In [5]:
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic.v1 import BaseModel, Field
from typing import List
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

In [6]:
temperature = 0.0

class BusinessName(BaseModel):
    name: str = Field(description="The name of the business")
    rating_score: float = Field(description="The rating score of the business. 0 is the worst, 10 is the best.")

class BusinessNames(BaseModel):
    names: List[BusinessName] = Field(description="A list of business names")

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=BusinessNames)

principles = """
- The name must be easy to remember.
- Use the {industry} industry and Company context to create an effective name.
- The name must be easy to pronounce.
- You must only return the name without any other text or characters.
- Avoid returning full stops, \n or any other characters.
- The maximum length of the name must be 10 characters.
"""

In [7]:
# Chat Model Output Parser:
model = ChatOpenAI() 
template = """Generate 5 business names for a new start up company in the {industry} industry. 
You must follow the following principles: {principles}
{format_instructions}
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

In [9]:
prompt_and_model = chat_prompt | model

result = prompt_and_model.invoke(
    {
        "principles": principles,
        "industry": "Data Science",
        "format_instructions": parser.get_format_instructions(),
    }
)
# The output parser, parses the LLM response into a Pydantic object
print(parser.parse(result.content))

names=[BusinessName(name='DataWave', rating_score=9.0), BusinessName(name='InfoMind', rating_score=8.0), BusinessName(name='AnalytiX', rating_score=8.0), BusinessName(name='DataSync', rating_score=7.0), BusinessName(name='Insighta', rating_score=7.0)]


In [10]:
chain = chat_prompt | model | parser

result = chain.invoke(
    {
        "principles": principles,   
        "industry": "Data Science",
        "format_instructions": parser.get_format_instructions(),
    }
)
print(result)

names=[BusinessName(name='DataWiz', rating_score=8.0), BusinessName(name='ByteIQ', rating_score=7.5), BusinessName(name='LogicLab', rating_score=8.2), BusinessName(name='InfoSync', rating_score=7.8), BusinessName(name='DataAce', rating_score=8.5)]


In [11]:
# Technically you can also write it like model() which uses the __call__ method. However the _call__ method is an older syntax.

# Or simply use model.invoke(input) whilst using .format_prompt():
chat_response_output = model.invoke(
    chat_prompt.format_prompt(
        principles=principles,
        industry="Data Science",
        format_instructions=parser.get_format_instructions(),
    ).to_messages()
)

pydantic_schema: BusinessNames = parser.parse(chat_response_output.content)
print(pydantic_schema, "\n")
print(pydantic_schema.json())

names=[BusinessName(name='DataWiz', rating_score=8.5), BusinessName(name='ByteIQ', rating_score=8.0), BusinessName(name='AlgoLab', rating_score=7.5), BusinessName(name='Insighta', rating_score=7.2), BusinessName(name='Statify', rating_score=7.8)] 

{"names": [{"name": "DataWiz", "rating_score": 8.5}, {"name": "ByteIQ", "rating_score": 8.0}, {"name": "AlgoLab", "rating_score": 7.5}, {"name": "Insighta", "rating_score": 7.2}, {"name": "Statify", "rating_score": 7.8}]}
